## Add abstract + Introduction about the project








## Packages required

In [ ]:
import os
import sys
import tensorflow as tf
import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import time



# Part pre-processing the data  (Very painful part..)








## 1.**Generate TFRecords files**

we are using TFRecords to define our dataset because of two key elements:
-First this is a tensorflow format so more efficient with tf, easier to work direclty with tf
-Since we work we large dataset TFRecords allows us to dereclty stream the data over the network, do not need to load it into the code.


## 2.**Processing the data**

All transformations that your dataset required






##### Basics functions 
those functions converting our data to feature for TFRecords files

In [ ]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))

def _bytes_features(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))

In [ ]:
def cut_edge(data):
    
    
    D,H,W,_ = data.shape
    D_s, D_e = 0, D-1
    H_s, H_e = 0, H-1
    W_s, W_e = 0, W-1
    
    
    while D_s < D:
        if data[D_s].sum() != 0:
            break
        D_s += 1
    
    while D_e > D_s:
        if data[D_e].sum() !=0:
            break
        D_e -=1
        
    while H_s < H:
        if data[:,H_s].sum() != 0:
            break
        H_s += 1
    
    while H_e > H_s:
        if data[:,H_e].sum() !=0:
            break
        H_e -=1
        
    while W_s < W:
        if data[:,:,W_s].sum() != 0:
            break
        W_s += 1
    
    while W_e > W_s:
        if data[:,:,W_e].sum() !=0:
            break
        W_e -=1
    
    original_shape=[D,H,W]
    cut_size = [int(D_s),int(D_e+1),int(H_s),int(H_e+1),int(W_s),int(W_e+1)]
    return(original_shape,cut_size)
            

In [ ]:
def load_subject(raw_data_dir,subject_id):
    
    
    subject_name = 'HeadAndNeck_anonymous-%d' %subject_id
    path_ct=os.path.join(raw_data_dir, subject_name+'.mha')
    
    img_ct = sitk.ReadImage(path_ct)
    img_ct_array = sitk.GetArrayFromImage(img_ct)
    D,H,W=img_ct_array.shape
    img_ct_array=np.resize(img_ct_array,[D-1,H,W])
    
    if subject_id < 17:
        path_ct_lbl=os.path.join(raw_data_dir, subject_name+'-label.mha')
        lbl_ct = sitk.ReadImage(path_ct_lbl)
        label_ct_array = sitk.GetArrayFromImage(lbl_ct)
        D,H,W=label_ct_array.shape
        label_ct_array=np.resize(label_ct_array,[D-1,H,W])
    else:
        label_ct_array=None
    
    return[img_ct_array[...,np.newaxis],label_ct_array[...,np.newaxis]]


In [ ]:
def prepare_validation(cutted_image, patch_size, overlap_stepsize):
    
    patch_ids = []
    
    D,H,W,_ = cutted_image.shape
    
    drange = list(range(0,D-patch_size+1, overlap_stepsize))
    hrange = list(range(0,H-patch_size+1, overlap_stepsize))
    wrange = list(range(0,W-patch_size+1, overlap_stepsize))
    
    
    if (D-patch_size) % overlap_stepsize != 0:
        drange.append(D-patch_size)

    if (H-patch_size) % overlap_stepsize != 0:
        hrange.append(H-patch_size)

    if (W-patch_size) % overlap_stepsize != 0:
        wrange.append(W-patch_size)
        
    
    for d in drange:
        for h in hrange:
            for w in wrange:
                patch_ids.append((d,h,w))
                
                
    return(patch_ids)

In [ ]:
def write_training_examples(ct_img,ct_lbl, original_shape, cut_size, output_file):
    
    
    writer = tf.python_io.TFRecordWriter(output_file)
    example= tf.train.Example(features=tf.train.Features(
    feature={'ct':_bytes_features([ct_img[:,:,:,0].tostring()]),
             'label':_bytes_features([ct_lbl[:,:,:,0].tostring()]),
             'cut_size':_int64_feature(cut_size),
             'original_shape':_int64_feature(original_shape),
                
    }
    
    ))
    
    
    writer.write(example.SerializeToString())
    writer.close()
    


In [ ]:
def write_validation_examples(ct_img, lbl_img, patch_size , cut_size, overlap_stepsize, output_file):
    
    
    ct_img=ct_img[cut_size[0]:cut_size[1],cut_size[2]:cut_size[3],cut_size[4]:cut_size[5],:]
    lbl_img=lbl_img[cut_size[0]:cut_size[1],cut_size[2]:cut_size[3],cut_size[4]:cut_size[5],:]
    
    patch_ids = prepare_validation(ct_img,patch_size,overlap_stepsize)
    print('Number of patches:', len(patch_ids))
    
    
    writer = tf.python_io.TFRecordWriter(output_file)
    
    for i in range(len(patch_ids)):
        (d,h,w) = patch_ids[i]
        
        _ct_img = ct_img[d:d+patch_size,h:h+patch_size,w:w+patch_size,:]
        _lbl_img = lbl_img[d:d+patch_size,h:h+patch_size,w:w+patch_size,:]
        
        
        example = tf.train.Example(features=tf.train.Features(
        feature={'ct':_bytes_features([_ct_img[:,:,:,0].tostring()]),
                 'label':_bytes_features([_lbl_img[:,:,:,0].tostring()])
                 
                 
            
        }
        ))
        
        writer.write(example.SerializeToString())
    
    writer.close()
        
    

    
    

In [ ]:
def write_prediction_examples(ct_img,patch_size, cut_size, overlap_stepsize, output_file):
    
    ct_img=ct_img[cut_size[0]:cut_size[1],cut_size[2]:cut_size[3],cut_size[4]:cut_size[5],:]
    
    patch_ids = prepare_validation(ct_img, patch_size, overlap_stepsize)
    print('Number of patches',len(patch_ids))
    
    writer = tf.python_io.TFRecordWriter(output_file)
    
    for i in range(len(patch_ids)):
        (d,h,w) = patch_ids[i]
        
        _ct_img = ct_img[d:d+patch_size,h:h+patch_size,w:w+patch_size,:]
        
        
        
        example = tf.train.Example(features=tf.train.Features(
        feature={'ct':_bytes_features([_ct_img[:,:,:,0].tostring()]) 
            
        }
        ))
        
        writer.write(example.SerializeToString())
    
    writer.close()
        
    
    
    
    

In [ ]:
def generate_files(raw_data_dir, output_path,valid_id, pred_id, patch_size,overlap_stepsize):
    number_of_patients=8
    
    if valid_id not in range(1,number_of_patients+1) and valid_id !=-1:
        print('the valid_id should be between [1, %d]' %number_of_patients)
        sys.exit(-1)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
        
    for i in range (1,number_of_patients+1):
        print('----Process subject %d:---' % i)
        
        subject_name = 'subject-%d' %i
        train_filename = os.path.join(output_path, subject_name+'.tfrecords')
        
        pred_subject_name = 'subject-%d-pred-%d-patch-%d' % (pred_id, overlap_stepsize, patch_size)
        pred_filename = os.path.join(output_path, pred_subject_name+'.tfrecords')
        
        valid_subject_name = 'subject-%d-valid-%d-patch-%d' %  (valid_id, overlap_stepsize, patch_size)
        valid_filename = os.path.join(output_path, valid_subject_name+'.tfrecords')
        
        converted_label_filename = 'subject-%d-label.npy' % valid_id
        converted_label_filename = os.path.join(output_path, converted_label_filename)
        
        if (i < number_of_patients+1  and not os.path.isfile(train_filename)) or \
            (i == pred_id and not os.path.isfile(pred_filename)) or \
            (i == valid_id and (not os.path.isfile (valid_filename) or \
                not os.path.isfile(converted_label_filename))):
                print('Loading data...')
                
                [ct,lbl] = load_subject(raw_data_dir, i)
                
                if lbl is not None:
                    print('Check label', np.unique(lbl),'Total number of labels',len(np.unique(lbl)))
                    
                    
                [original_shape, cut_size] = cut_edge(ct)
                print('check original size', original_shape)
                print('check cut size', cut_size)
                
        
        if not os.path.isfile(train_filename) and i < number_of_patients+1:
            print('create the training file')
            write_training_examples(ct,lbl,original_shape,cut_size, train_filename)
            
        if i == valid_id:
            if not os.path.isfile(valid_filename):
                print('Create the validation file:')
                write_validation_examples(ct,lbl, patch_size, cut_size, overlap_stepsize ,valid_filename)
                
            if not os.path.isfile(pred_filename):
                print('Create the converted label file:')
                np.save(converted_label_filename, lbl[:,:,:,0])
                
        if i == pred_id:
            if not os.path.isfile(pred_filename):
                print('Create the prediction file')
                write_prediction_examples(ct,patch_size,cut_size,overlap_stepsize, pred_filename)
                
        print('---Done---')
                
        
                
        

                
        

        

In [ ]:
raw_data_dir=r"C:\Users\r778e\Desktop\Data_to_aug\data_set_no_augmentation"
output_path=r"C:\Users\r778e\Desktop\tfrecord"
valid_id=3
pred_id=5
patch_size=32
overlap_stepsize=20
generate_files(raw_data_dir, output_path,valid_id, pred_id, patch_size,overlap_stepsize)

In [ ]:
def get_filenames(data_dir,mode,valid_id, pred_id, overlap_step, patch_size):
    number_of_patients=8
    if mode == 'train':
        train_files = [os.path.join(data_dir, 'subject-%d.tfrecords' % i) for i in range(1, number_of_patients+1) if i != valid_id]
        
        
        for f in train_files:
            assert os.path.isfile(f), \
            ('Run generate_tfrecord.py to generate training files.')
        return train_files
    elif mode == 'valid':
        valid_file = os.path.join(data_dir, 'subject-%d-valid-%d-patch-%d.tfrecords' % (valid_id, overlap_step, patch_size ))
        assert os.path.isfile(valid_file), \
            ('Check generate_tfrecords function to generate the validation file')
        return [valid_file]
    elif mode == 'pred':
        pred_file = os.path.join(data_dir, 'subject-%d-pred-%d-patch-%d.tfrecords' %(pred_id, overlap_step, patch_size))
        assert os.path.isfile(pred_file), \
           ('Check generate_tfrecords function to generate the prediction file')
        return [pred_file]
    

In [ ]:
def decode_train(serialized_example):
    
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'ct':tf.FixedLenFeature([],tf.string),
                                       'label':tf.FixedLenFeature([],tf.string),
                                          'cut_size':tf.FixedLenFeature(6,tf.int64),
                                          'original_shape':tf.FixedLenFeature(3,tf.int64)
     
                                      
                                      })
    
    img_shape = features['original_shape']
    cut_size = features['cut_size']
    
    image_ct=tf.decode_raw(features['ct'],tf.int16)
    image_ct = tf.reshape(image_ct, img_shape)
    
    
    label_ct=tf.decode_raw(features['label'],tf.uint16)
    label_ct = tf.reshape(label_ct, img_shape)

    
    
    image_ct= tf.cast(image_ct, tf.float32)
    label_ct= tf.cast(label_ct, tf.float32)

    
    return image_ct, label_ct,cut_size


In [ ]:
def decode_valid(serialized_example):
    patch_size=32
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'ct':tf.FixedLenFeature([],tf.string),
                                       'label':tf.FixedLenFeature([],tf.string),
                                     
            
                                      })
    
    patch_shape=[patch_size]*3
    image_ct=tf.decode_raw(features['ct'],tf.int16)
    image_ct = tf.reshape(image_ct, patch_shape)
  
    label_ct=tf.decode_raw(features['label'],tf.uint16)
    label_ct = tf.reshape(label_ct, patch_shape)

    
    
    image_ct= tf.cast(image_ct, tf.float32)
    label_ct= tf.cast(label_ct, tf.float32)

    
    return image_ct, label_ct


In [ ]:
def decode_pred(serialized_example):
    patch_size=32
    features = tf.parse_single_example(serialized_example,
                                      features={
                                          'ct':tf.FixedLenFeature([],tf.string),

        
                                      
                                    })
    
    
    patch_shape=[patch_size]*3
    
    image_ct=tf.decode_raw(features['ct'],tf.int16)
    image_ct = tf.reshape(image_ct, patch_shape)
    

    
    image_ct= tf.cast(image_ct, tf.float32)
    label_ct = tf.zeros(image_ct.shape)



    
    return image_ct, label_ct


In [ ]:
def crop_image(image_ct,label_ct,cut_size):
    
    patch_size=32
    data= tf.stack([image_ct,label_ct],axis=-1)
    
    
    image = tf.random_crop(data[cut_size[0]:cut_size[1], cut_size[2]:cut_size[3], cut_size[4]:cut_size[5],:],[patch_size,patch_size,patch_size,2])

    [image_ct, lbl_image] = tf.unstack(image,2,axis=-1)
    
    return image_ct, lbl_image


    

In [ ]:
def normalize_image(image_ct,label_ct):
    
    image_ct = tf.image.per_image_standardization(image_ct)
  
    
    feature = tf.cast(image_ct, tf.float32)
    label = tf.cast(label_ct, tf.int32)
    
    return feature,label



In [ ]:
def input_function(data_dir, mode,valid_id, pred_id ,overlap_step , patch_size, batch_size, buffer_size ,num_epochs=1, num_parallel_calls=1):
    
    
    
    with tf.name_scope('input'):
        
        filenames = get_filenames(data_dir, mode, valid_id, pred_id, overlap_step, patch_size)
        dataset = tf.data.TFRecordDataset(filenames)
        
        
        dataset = dataset.prefetch(buffer_size=batch_size)
        
        
        if mode == 'train':
            
            dataset = dataset.shuffle(buffer_size=buffer_size)
            
            
            
        dataset = dataset.repeat(num_epochs)
        
        if mode == 'train':
            dataset = dataset.map(decode_train, num_parallel_calls=num_parallel_calls)
            dataset = dataset.map(crop_image, num_parallel_calls=num_parallel_calls)
            
        elif mode == 'valid':
            dataset = dataset.map(decode_valid, num_parallel_calls=num_parallel_calls)
        elif mode == 'pred':
            dataset = dataset.map(decode_pred , num_parallel_calls=num_parallel_calls)
        
        
        dataset = dataset.map(normalize_image, num_parallel_calls=num_parallel_calls)
        dataset = dataset.batch(batch_size)
        
        
        dataset.prefetch(1)
        
        iterator = dataset.make_one_shot_iterator()
        feature, label = iterator.get_next()
        
        return tf.expand_dims(feature,-1), tf.cast(label, tf.int32)


## Definition of operations that will be use to built the differents blocks

### All operations defined are axecuted on 5D tensor  


In [ ]:


################################################################################

# Basic operations building the network

################################################################################



def Pool3d(inputs, kernel_size, strides):

	"""Performs 3D max pooling."""



    
	return tf.layers.max_pooling3d(

			inputs=inputs,

			pool_size=kernel_size,

			strides=strides,

			padding='same')





def Deconv3D(inputs, filters, kernel_size, strides, use_bias=False):

	"""Performs 3D deconvolution without bias and activation function."""



	return tf.layers.conv3d_transpose(

			inputs=inputs,

			filters=filters,

			kernel_size=kernel_size,

			strides=strides,

			padding='same',

			use_bias=use_bias,

			kernel_initializer=tf.truncated_normal_initializer())





def Conv3D(inputs, filters, kernel_size, strides, use_bias=False):

	"""Performs 3D convolution without bias and activation function."""



	return tf.layers.conv3d(

			inputs=inputs,

			filters=filters,

			kernel_size=kernel_size,

			strides=strides,

			padding='same',

			use_bias=use_bias,

			kernel_initializer=tf.truncated_normal_initializer())





def Dilated_Conv3D(inputs, filters, kernel_size, dilation_rate, use_bias=False):

	"""Performs 3D dilated convolution without bias and activation function."""



	return tf.layers.conv3d(

			inputs=inputs,

			filters=filters,

			kernel_size=kernel_size,

			strides=1,

			dilation_rate=dilation_rate,

			padding='same',

			use_bias=use_bias,

			kernel_initializer=tf.truncated_normal_initializer())





def BN_ReLU(inputs, training):

	"""Performs a batch normalization followed by a ReLU6."""



	# We set fused=True for a significant performance boost. See

	# https://www.tensorflow.org/performance/performance_guide#common_fused_ops

	inputs = tf.layers.batch_normalization(

				inputs=inputs,

				axis=-1,

				momentum=0.997,

				epsilon=1e-5,

				center=True,

				scale=True,

				training=training, 

				fused=True)



	return tf.nn.relu6(inputs)

## Network implementation 

In [ ]:
class Network(object):
    
    def __init__(self,num_classes,num_filters,network_depth,block_sizes,block_strides):
        
        self.num_classes = num_classes
        
        self.num_filters = num_filters
        self.block_sizes = [1] * block_sizes
        self.block_strides = [1] + [2] * (block_strides - 1)
        
    
    
    
    def __call__(self, inputs, training):

        
        
        return self._build_network(inputs, training, self.block_sizes, self.block_strides)
    
    def _build_network(self,inputs, training, block_sizes, block_strides):

        
        inputs = Conv3D(inputs=inputs,
                       filters=self.num_filters,
                       kernel_size=3,
                       strides=1)
        
        inputs = tf.identity(inputs,'initial_conv')
        
        skip_inputs = []
        
        for i, num_blocks in enumerate(self.block_sizes):
            
            
            num_filters = self.num_filters * (2**i)
            inputs = self._encoding_block_layer (
                                                inputs=inputs, 
                                              filters=num_filters,
                                                block_fn=self._residual_block,
                                             blocks=num_blocks,
                                     strides=self.block_strides[i],
                                         training=training,
                                          name='encode_block_layer{}'.format(i+1))
                
            skip_inputs.append(inputs)
        
        
        for i, num_blocks in reversed(list(enumerate(self.block_sizes[1:]))):
            num_filters = self.num_filters * (2**i)
            inputs = self._decoding_block_layer(inputs=inputs,
                                                    skip_inputs=skip_inputs[i],
                                                    filters=num_filters,
                                                    block_fn=self._residual_block,
                                                    blocks=1,
                                                    strides=self.block_strides[i+1],
                                                    training=training,
                                                    name='decode_block_layer{}'.format(len(self.block_sizes)-i-1))
        
        inputs= self._output_block_layer(inputs=inputs, training=training)
        
                
        return inputs 
                
            
    def _output_block_layer(self,inputs,training):
        
        inputs = BN_ReLU(inputs, training)
                
        inputs = tf.layers.dropout(inputs,rate=0.5, training=training)
                
        inputs = Conv3D( inputs=inputs,
                               filters= self.num_classes,
                               kernel_size=1,
                               strides=1,
                               use_bias=True)
                
                
        return tf.identity(inputs, 'output')

                
                
                
    def _encoding_block_layer(self, inputs,filters, block_fn,blocks, strides,training,name):
        
        def projection_shortcut(inputs):
            return Conv3D(       inputs=inputs,
                                 filters=filters,
                                 kernel_size=1,
                                 strides=strides   ) 
        inputs = block_fn(inputs, filters, training, projection_shortcut, strides)
                
        for _ in range(1,blocks):
            inputs = block_fn(inputs,filters, training, None, 1)
                
                
        return tf.identity(inputs, name)
                
                
                
    def _decoding_block_layer(self, inputs, skip_inputs, filters,block_fn,blocks, strides, training, name):
        inputs = Deconv3D(inputs=inputs,
                                 filters=filters,
                                 kernel_size=3,
                                 strides=strides)
                
        inputs = inputs + skip_inputs
                
                
        for _ in range(0, blocks):
            inputs = block_fn(inputs, filters, training, None,1)
                
        return tf.identity(inputs,name)

    def _residual_block(self, inputs, filters, training,
                        projection_shortcut, strides):
        shortcut = inputs
        inputs = BN_ReLU(inputs, training)
                
                
                
        if projection_shortcut is not None:
            shortcut = projection_shortcut(inputs)
                
                
        inputs = Conv3D(inputs=inputs,
                               filters=filters,
                               kernel_size=3,
                               strides=strides)
                
        inputs = BN_ReLU(inputs, training)
                
        inputs = Conv3D(inputs=inputs,
                               filters=filters,
                               kernel_size=3,
                               strides=1)
                
                
        return inputs + shortcut 
                

                
                
                
                
                
            
                
                
                
                
            
                
                
            
                
            
                
                
                
    
                

In [ ]:
def _model_fn(features, labels, mode):
    
    num_classes=39
    num_filters=32
    network_depth=3
    block_sizes=3
    block_strides=3
    Training=True
    
    
    net = Network(num_classes,num_filters,network_depth,block_sizes,block_strides)
    logits = net(features, Training)

    predictions = {
        'classes' : tf.argmax(logits, axis=-1),
        'probabilities':tf.nn.softmax(logits, name='softmax_tensor')}
    print('We in')


    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)



    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels, logits=logits))


    tf.identity(cross_entropy, name='cross_entropy')
    tf.summary.scalar('cross_entropy',cross_entropy)


    loss = cross_entropy + 2e-6 * tf.add_n([tf.nn.l2_loss(v) for v in tf.trainable_variables() if 'kernel' in v.name])
    print('we in')

    if mode == tf.estimator.ModeKeys.TRAIN:
        
        print('We in')
        global_step = tf.train.get_or_create_global_step()


        optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss, global_step)

    else:
        train_op = None

    accuracy = tf.metrics.accuracy(labels, predictions['classes'])
    metrics = {'accuracy':accuracy}


    tf.identity(accuracy[1],name='train_accuracy')
    tf.summary.scalar('train_accuracy', accuracy[1])
    print(metrics)


    return tf.estimator.EstimatorSpec(mode=mode,
                                     predictions=predictions,
                                     loss=loss,
                                     train_op=train_op,
                                     eval_metric_ops=metrics)



## Training step / Validation step (optional)

## Setting parameters for training and validation

In [ ]:
num_classes=39 #the number of classes
network_depth=3 #the network depth
num_filters=32 #the initial numbers of filter for the first convolution block
block_sizes=3 
block_strides=3
tf.logging.set_verbosity(tf.logging.INFO)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'#enable gpu 
os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
num_training_subs=6
train_epochs=100000
epochs_per_eval=5000
data_dir=r"C:\Users\r778e\Desktop\tfrecord" #the directory where the input data is stored as TFRecords file
model_dir=r'C:\Users\r778e\Desktop\modelsave_test'#the directory where you the model is saved
batch_size=5
validation_id=3 #the patient used for the validation
num_parallel_callszer=1 
overlap_step=20
patch_size=32

In [ ]:
mode=tf.estimator.ModeKeys.TRAIN 
training=True


save_checkpoints_steps = 5000 * \
                 num_training_subs // batch_size

run_config = tf.estimator.RunConfig().replace(
    save_checkpoints_steps=save_checkpoints_steps,
    keep_checkpoint_max=0)


classifer = tf.estimator.Estimator(
    model_fn=_model_fn,
    model_dir=model_dir,
    config=run_config
)


for _ in range(train_epochs // epochs_per_eval):
    tensor_to_log = {
        'cross_entropy':'cross_entropy',
        'train_accuracy':'train_accuracy'
    }

    logging_hook = tf.train.LoggingTensorHook(tensors=tensor_to_log, every_n_iter=100)

    print('Starting a training cycle zerrrrr!')

    def input_fn_train():
        return input_function( data_dir=data_dir,
                               mode='train',
                               patch_size=patch_size, 
                                    batch_size=batch_size,
                                    buffer_size=num_training_subs,
                                    valid_id=validation_id,
                                    pred_id=-1, # not used
                                    overlap_step=-1, # not used
                                    num_epochs=epochs_per_eval,
                                    num_parallel_calls=num_parallel_callszer)

    classifer.train(input_fn=input_fn_train, hooks=[logging_hook])
    
    
#    if self.conf.validation_id != -1:
#        print('Starting to evaluate.')
#
#        def input_fn_eval():
#            return input_function(
#            data_dir=data_dir,
#            mode='valid',
#            patch_size=patch_size,
#            batch_size=batch_size,
#            buffer_size=-1, # not used
#            valid_id=validation_id,
#            pred_id=-1, # not used
#            overlap_step=overlap_step,
#            num_epochs=1,
#            num_parallel_calls=num_parallel_calls)
#
#        classifier.evaluate(input_fn=input_fn_eval)





## Prediction step

In [ ]:
def predict(raw_data_dir,data_dir,model_dir,num_parallel_calls,save_dir,checkpoint_num,prediction_id):
    os.environ['TF_ENABLE_WINOGRAD_NONFUSED'] = '1'
    mode=tf.estimator.ModeKeys.PREDICT
    
    print(50*'-')
    print('Perform prediction for subject-%d:' % prediction_id)

    print('loading data...')
    print(50*'-')    

    [ct,_] = load_subject(raw_data_dir, prediction_id)

    (_,cut_size) = cut_edge(ct)
    print('Check cut_size:', cut_size)

    cutted_CT = ct[cut_size[0]:cut_size[1],cut_size[2]:cut_size[3],cut_size[4]:cut_size[5],:]
    patch_ids = prepare_validation(cutted_CT,patch_size,overlap_step)
    num_patchs = len(patch_ids)
    print('Numbers of patches:', num_patchs)


    print('Initialize')
    classifier = tf.estimator.Estimator(
        model_fn=_model_fn,
        model_dir=model_dir)



    def input_fn_predict():
            return input_function( data_dir=data_dir,
                                    mode='pred',
                                    patch_size=patch_size,
                                    batch_size=batch_size,
                                    buffer_size=-1,
                                    valid_id=-1,
                                    pred_id=prediction_id, # not used
                                    overlap_step=overlap_step, # not used
                                    num_epochs=1,
                                    num_parallel_calls=num_parallel_calls)


    checkpoint_file = os.path.join(model_dir,'model.ckpt-'+str((checkpoint_num)))


    preds = classifier.predict(
            input_fn=input_fn_predict,
        checkpoint_path=checkpoint_file)


    print('Starting to predict')


    predictions = {}

    for i, pred in enumerate(preds):
            location = patch_ids[i]

            print('Step {:d}/{:d} processing results for ({:d},{:d},{:d})'.format(

            i+1, num_patchs, location[0], location[1], location[2]),
            end='\r',

            flush=True)

            logits = pred['probabilities']

            for j in range(patch_size):

                    for k in range(patch_size):

                            for l in range(patch_size):    

                                    key = (location[0]+j, location[1]+k, location[2]+l)

                                    if key not in predictions.keys():

                                                predictions[key] = []

                                    predictions[key].append(logits[j, k, l, :])



    print('Averaging results...')



    results = np.zeros((ct.shape[0], ct.shape[1], ct.shape[2], num_classes),

    dtype=np.float32)

    print(results.shape)

    for key in predictions.keys():
        results[cut_size[0]+key[0],cut_size[2]+key[1], cut_size[4]+key[2]] = np.mean(predictions[key], axis=0)

    results = np.argmax(results, axis=-1)



    print('Saving results...')



    if not os.path.exists(save_dir):

            os.makedirs(save_dir)

    save_filename = 'preds-' + str(checkpoint_num) + '-sub-' + str(prediction_id) + '-overlap-' + str(overlap_step) + '-patch-' + str(patch_size) + '.npy'

    save_file = os.path.join(save_dir, save_filename)

    np.save(save_file, results)



    print('Done.')


 

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
raw_data_dir=r"C:\Users\r778e\Desktop\Data_to_aug\data_set_no_augmentation" #the directory where the raw data is stored
data_dir=r"C:\Users\r778e\Desktop\tfrecord" #the directory where the input data is stored as TFRecords file
model_dir=r'C:\Users\r778e\Desktop\KnowUbone\model_save_64_patch_16_overlap_1' #the directory where you saved your model
num_parallel_calls=1
save_dir=r'C:\Users\r778e\Desktop\save prediction' #The directory where you want to save your prediction
checkpoint_num=60000
prediction_id=5
predict(raw_data_dir,data_dir,model_dir,num_parallel_calls,save_dir,checkpoint_num,prediction_id)

## load the npa prediction and convert to mha

In [ ]:
def load_prediction(save_dir,prediction_id,overlap_step,patch_size,checkpoint_num):
    save_filename = 'preds-' + str(checkpoint_num) + '-sub-' + str(prediction_id) + '-overlap-' + str(overlap_step) + '-patch-' + str(patch_size) + '.npy'
    save_file = os.path.join(save_dir, save_filename)
    prediction=np.load(save_file)
    return prediction


In [ ]:
prediction=load_prediction(save_dir,prediction_id,overlap_step,patch_size,checkpoint_num)


In [ ]:
def convert_npa_to_mha_and_npa(prediction,id_patient,output_dir):
    
    """Purpose: convert the npa prediction to the conform mha file.
    
        Args:
            prediction:
            id_patient:
            outputdir_dir:
    
        
        
    """
    
    npa_res=prediction.astype('int16')
    res_img = sitk.GetImageFromArray(npa_res)
    
    npa_zeros=np.zeros([512,512],'int16')
    npa_zeros=npa_zeros[np.newaxis,:,:]
    
    filename_ground_truth=r"D:\xxx_gathering_augmented\HeadAndNeck_anonymous-%i.mha" %id_patient
    ground_truth = sitk.ReadImage(filename_ground_truth)
    
    if len(res_img) == ground_truth.GetDepth():
        res_img.CopyInformation(ground_truth)
        sitk.WriteImage(res_img,os.path.join(output_dir,"prediction_from_patient_%i.mha"%id_patient))
        
        print('Done.')
        print('The prediction has been succesfuly converted to mha file')
    
    else:
        npa_res=np.concatenate((npa_res,npa_zeros),axis=0)
        res_img = sitk.GetImageFromArray(npa_res)
        res_img.CopyInformation(ground_truth)
        sitk.WriteImage(res_img,os.path.join(output_dir,"prediction_from_patient_%i.mha"%id_patient))
       
        print('Done.')
        print('The prediction has been succesfuly converted to mha file and npa file')
    

In [ ]:
id_patient=5
output_dir=""
convert_to_mha(prediction,id_patient,output_dir)